# Experts Hook for Mixtral 8x7B

This experiment is storing all the tokens of a datasets into a database where I can view which tokens go to which experts and plot paths. Maybe given multiple datasets, I could see how they differ.

In [1]:
import torch as t
import pandas as pd
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import altair as alt

import os
from collections import defaultdict
from dotenv import load_dotenv
from huggingface_hub import login

In [2]:
load_dotenv()
login(os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


## Dataset 

We will use the `Salesforce/wikitext` dataset. Its very simple, unassuming and easily understand.  

In [17]:
dataset = load_dataset("Salesforce/wikitext", 'wikitext-2-raw-v1', split="test")
data = "\n".join(dataset["text"][:8])

## Model Setup

We would need to add a forward hook into the layers to extract which experts a token gets send to which experts. To store the information, we will use 2 simple databases right now. 
- expert_to_token: 32 keys representing the Layers, each contain a dictionary of where keys are the exprts and values are sets of tokens. Tracks which tokens are sent to the expert.
- token_to_expert: keys are tokens, values are 32-length lists of 2-tuples. Tracks which experts processed a token through the decoder layer. 

In [4]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=t.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModel.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [18]:
NUM_LAYERS = len(model.layers)
token_to_expert = defaultdict(list)
expert_to_token = {f"Layer {l}": defaultdict(set) for l in range(NUM_LAYERS)}

In [19]:
print(model)

MixtralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MixtralDecoderLayer(
      (self_attn): MixtralSdpaAttention(
        (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MixtralRotaryEmbedding()
      )
      (block_sparse_moe): MixtralSparseMoeBlock(
        (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
        (experts): ModuleList(
          (0-7): 8 x MixtralBlockSparseTop2MLP(
            (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (w3): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (act_fn): SiLU()
        

Right now, we just define a hook that prints out which experts were chose for a specific output of the attention heads. We need to understand which tokens are directed where. So I will try and figure out which ones go where.

In [23]:
def get_print_token(prompt: str, layer_num:int):
    ids = tokenizer.encode(prompt, add_special_tokens=False) 
    tokens = tokenizer.convert_ids_to_tokens(ids)
    print(len(tokens))
    pos_to_token = {i: token for i, token in enumerate(tokens)}

    def print_tokens(module, input, output):
        _, topk_index = t.topk(output, 2, dim=1)
        
        # topk_list of of the shape [S_l, 2] where S_l is the length of the sequence
        topk_list = topk_index.tolist()
        print(topk_list)
        for i, topk in enumerate(topk_list):
            token = pos_to_token[i]
            topk = tuple(topk)
            token_to_expert[token].append(topk)
            expert_1, expert_2 = topk
            expert_to_token[f"Layer {layer_num}"][expert_1].add(token)
            expert_to_token[f"Layer {layer_num}"][expert_2].add(token)
    return print_tokens


In [24]:
hooks = []
for layer_num, layers in enumerate(model.layers):
    hook = layers.block_sparse_moe.gate.register_forward_hook(get_print_token(data, layer_num))
    hooks.append(hook)

494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494


In [22]:
tokenized = tokenizer(data, return_tensors="pt", add_special_tokens=False)

with t.no_grad():
  outputs = model(**tokenized.to(model.device), output_hidden_states=True, return_dict=True)

print(outputs.keys())

[[6, 2], [5, 0], [5, 6], [2, 5], [1, 0], [2, 5], [0, 4], [1, 0], [2, 5], [0, 5], [5, 6], [5, 6], [5, 2], [1, 5], [2, 5], [0, 4], [1, 4], [6, 4], [3, 2], [1, 7], [1, 0], [3, 5], [1, 0], [3, 7], [1, 7], [0, 7], [3, 5], [3, 2], [4, 5], [3, 2], [1, 0], [5, 6], [3, 5], [5, 3], [3, 5], [4, 5], [1, 0], [5, 7], [2, 5], [1, 0], [0, 1], [2, 5], [1, 2], [5, 4], [2, 5], [6, 7], [4, 7], [4, 7], [4, 7], [3, 5], [6, 5], [6, 4], [2, 0], [5, 7], [3, 5], [3, 1], [4, 7], [1, 0], [5, 4], [2, 6], [0, 4], [2, 5], [1, 4], [4, 0], [5, 2], [1, 0], [1, 4], [0, 1], [3, 5], [6, 5], [6, 5], [4, 7], [5, 4], [2, 5], [6, 7], [4, 7], [4, 7], [4, 5], [7, 5], [2, 5], [1, 7], [1, 7], [1, 7], [3, 5], [3, 5], [4, 5], [3, 5], [1, 5], [1, 0], [5, 4], [2, 5], [1, 0], [1, 0], [1, 0], [1, 0], [2, 6], [4, 7], [5, 4], [2, 5], [6, 7], [4, 7], [4, 7], [7, 4], [3, 5], [5, 6], [2, 5], [6, 7], [4, 7], [4, 7], [4, 7], [2, 5], [0, 4], [1, 7], [4, 1], [3, 2], [0, 7], [7, 0], [2, 5], [1, 0], [2, 5], [5, 2], [2, 6], [0, 1], [2, 5], [2, 6],

KeyError: 11

In [16]:
expert_to_token

{'Layer 0': defaultdict(set,
             {6: {'<0x0A>', '▁', '▁='},
              2: {'oul', '▁', '▁Robert'},
              5: {'<0x0A>', 'oul', '▁', '▁=', '▁Robert'},
              0: {'<0x0A>', 'ter', '▁=', '▁B'},
              1: {'▁=', '▁B'},
              4: {'ter'}}),
 'Layer 1': defaultdict(set,
             {3: {'▁'},
              0: {'▁', '▁='},
              2: {'<0x0A>', 'oul', 'ter', '▁', '▁=', '▁B'},
              4: {'<0x0A>', 'oul', '▁=', '▁B'},
              5: {'▁Robert'},
              7: {'▁Robert'},
              1: {'<0x0A>', 'ter'}}),
 'Layer 2': defaultdict(set,
             {7: {'<0x0A>', '▁'},
              1: {'▁'},
              0: {'<0x0A>', 'oul', 'ter', '▁', '▁=', '▁B', '▁Robert'},
              2: {'<0x0A>', '▁=', '▁Robert'},
              6: {'▁='},
              4: {'oul', '▁', '▁B'},
              5: {'ter'}}),
 'Layer 3': defaultdict(set,
             {6: {'ter', '▁'},
              4: {'oul', '▁'},
              3: {'<0x0A>', '▁='},
              1

In [ ]:
token_to_expert

In [11]:
for handle in hooks:
    handle.remove()